# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [47]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
weather_data = pd.read_csv("../output_data/cities.csv")
data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Tilichiki,60.4667,166.1000,58.59,63,71,15.52,RU,1627425893
1,1,Hermanus,-34.4187,19.2345,55.80,75,88,15.70,ZA,1627425434
2,2,Isangel,-19.5500,169.2667,75.31,69,40,3.44,VU,1627425895
3,3,Hobart,-42.8794,147.3294,55.26,82,40,5.01,AU,1627425435
4,4,Tegul'det,57.3064,88.1667,50.85,91,29,0.13,RU,1627425898


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight.
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]

In [35]:
# Create map
map_fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)
# Add layer
map_fig.add_layer(heat_layer)

# Display figure
map_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [42]:
# Narrow down data to my perfect vacation weather conditions: temp >= 75, cloudiness <= 15%, and humidity between 30% and 50%
vacation_cities = weather_data.loc[(weather_data["Max Temp"] >= 75) & (weather_data["Cloudiness"] <= 15) &
                                   (weather_data["Humidity"] >= 30) & (weather_data["Humidity"] <= 50)].dropna()
vacation_cities.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
138,138,São Félix do Xingu,-6.6447,-51.9950,79.14,38,0,1.07,BR,1627426073
310,310,Corinth,37.9407,22.9573,87.69,42,0,1.97,GR,1627426292
374,374,Hibbing,47.4272,-92.9377,83.59,31,0,3.56,US,1627426373
388,388,Taos,36.4072,-105.5731,79.20,36,1,3.44,US,1627426391
543,543,Sterling,39.0062,-77.4286,94.33,40,1,1.01,US,1627426398


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [46]:
# Store into hotel_df variable and keep necessary columns
hotel_df = vacation_cities.loc[:,["City","Country", "Lat", "Lng"]]

# Add an empty column for "Hotel Name"
hotel_df["Hotel Name"]= ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
138,São Félix do Xingu,BR,-6.6447,-51.9950,
310,Corinth,GR,37.9407,22.9573,
374,Hibbing,US,47.4272,-92.9377,
388,Taos,US,36.4072,-105.5731,
543,Sterling,US,39.0062,-77.4286,
551,Nizip,TR,37.0097,37.7942,
563,Manavgat,TR,36.7867,31.4431,


In [53]:
# Set up url and parameters for api 
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

# Set up for loop to extract necessary info from api
for index, row in hotel_df.iterrows():
    # extract city name and location
    city = row["City"]
    lat = row["Lat"]
    lng = row["Lng"]
    
    # add lat and lng to params
    params["location"] = f"{lat},{lng}"
    
    # api request
    response = requests.get(url, params=params).json()
    print(f"Retrieving Hotel Results for: {city}.")
    
    # save results to a variable
    hotel_results = response["results"]
    
    # wrap in try statement 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_results[0]['name']
        print(f"Closest hotel in {city} is {hotel_results[0]['name']}.")
    except (KeyError, IndexError):
        print("No hotel found... skipping to next city.")
        
    # time delay
    time.sleep(1)
    
# Print statement when finished
print("-------Hotel Search Complete-------")


Retrieving Hotel Results for: São Félix do Xingu.
Closest hotel in São Félix do Xingu is Apart Hotel Rio Xingu.
Retrieving Hotel Results for: Corinth.
Closest hotel in Corinth is Hotel King Saron.
Retrieving Hotel Results for: Hibbing.
Closest hotel in Hibbing is Hampton Inn Hibbing.
Retrieving Hotel Results for: Taos.
Closest hotel in Taos is Kachina Lodge.
Retrieving Hotel Results for: Sterling.
Closest hotel in Sterling is Hampton Inn & Suites Herndon-Reston.
Retrieving Hotel Results for: Nizip.
Closest hotel in Nizip is BEĞER OTEL.
Retrieving Hotel Results for: Manavgat.
Closest hotel in Manavgat is Can Garden Beach Hotel.
-------Hotel Search Complete-------


In [54]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [56]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
map_fig.add_layer(markers)

# Display figure
map_fig

Figure(layout=FigureLayout(height='420px'))